<a href="https://colab.research.google.com/github/pankosmas/Arduino-Projects/blob/main/notebooks/Topic_Modeling_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cell 1: Install Necessary Libraries

In [ ]:
!pip install pymongo bertopic sentence-transformers umap-learn hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00


Cell 2: Import Libraries

In [ ]:
import re
from pymongo import MongoClient
from pymongo.errors import WriteError
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.cluster import HDBSCAN
from umap import UMAP
import pandas as pd
import os

(Optional) Cell 3: MongoDB Connection Setup &  Load Data from MongoDB

In [ ]:
# Replace with your connection string
mongo_client = MongoClient("mongodb://localhost:27017/")

# Replace with your database names
articles_db = mongo_client["dataset"]
speeches_db = mongo_client["prime_minister"]

# Replace with your collection names
articles_collection = articles_db["news_articles"]
speeches_collection = speeches_db["articles_speeches"]

# Convert MongoDB data to pandas DataFrames
articles_df = pd.DataFrame(list(articles_collection.find({}, {"_id": 0, "processed_text": 1})))
speeches_df = pd.DataFrame(list(speeches_collection.find({}, {"_id": 0, "processed_text": 1})))

# Ensure datasets are non-empty
if articles_df.empty or speeches_df.empty:
    raise ValueError("One or both collections are empty. Please check your MongoDB collections.")

KeyboardInterrupt: 

Cell 4: Alternative File Load

In [ ]:
# Flatten nested fields
from pandas import json_normalize

# Specify the file path in your Drive
speeches_file_path = '/content/drive/MyDrive/datasets/articles_speeches_fixed.json'
articles_file_path = '/content/drive/MyDrive/datasets/news_articles_fixed.json'

# Load the JSON file into a DataFrame
speeches_df = pd.read_json(speeches_file_path, lines=True)
articles_df = pd.read_json(articles_file_path, lines=True)

articles_df = articles_df[['processed_text', 'assigned_category', 'timestamp']]

# Check the first few rows
# print(articles_df.head())
# print(speeches_df.head())

KeyError: "None of [Index(['processed_text', 'assigned_category', 'timestamp'], dtype='object')] are in the [columns]"

Cell 5: Set Up Output Directory

In [ ]:
output_dir = "topic_modeling_results"
os.makedirs(output_dir, exist_ok=True)

Cell 6: Initialize SentenceTransformer Model

In [ ]:
embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Cell 7: Generate Embeddings

In [ ]:
print("Generating embeddings...")
articles_embeddings = embedding_model.encode(articles_df["processed_text"].tolist(), show_progress_bar=True, device="cuda")
speeches_embeddings = embedding_model.encode(speeches_df["processed_text"].tolist(), show_progress_bar=True, device="cuda")

Cell 8: Initialize UMAP and HDBSCAN

In [ ]:
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)

Cell 9: Train BERTopic for Articles

In [ ]:
print("Training BERTopic for articles...")
articles_topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False)
articles_topics = articles_topic_model.fit_transform(articles_embeddings)

Cell 10: Save Articles Model and Visualizations

In [ ]:
articles_topic_model.save(os.path.join(output_dir, "articles_topic_model.bertopic"))
articles_topic_model.visualize_barchart(top_n_topics=10).write_html(os.path.join(output_dir, "articles_topics_barchart.html"))
articles_topic_model.visualize_topics().write_html(os.path.join(output_dir, "articles_topics_interactive.html"))

Cell 11: Train BERTopic for Speeches

In [ ]:
print("Training BERTopic for speeches...")
speeches_topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False)
speeches_topics = speeches_topic_model.fit_transform(speeches_embeddings)

Cell 12: Save Speeches Model and Visualizations

In [ ]:
speeches_topic_model.save(os.path.join(output_dir, "speeches_topic_model.bertopic"))
speeches_topic_model.visualize_barchart(top_n_topics=10).write_html(os.path.join(output_dir, "speeches_topics_barchart.html"))
speeches_topic_model.visualize_topics().write_html(os.path.join(output_dir, "speeches_topics_interactive.html"))

Cell 13: Completion Message

In [ ]:
print("Topic modeling completed with optimizations. All models and visualizations saved.")